## Independent Project: Converting Election Data from NBC into a CSV File

While looking for project ideas on Kaggle, [this dataset](https://www.kaggle.com/imoore/2020-us-general-election-turnout-rates/activity) caught my eye. My first instinct was to investigate if there was a correlation between the turnout in a state, and the share of votes that Joe Biden or Donald Trump recieved there. 

However, a Google search for '2020 Election Results CSV' didn't yield any results. That's frustrating, given that several news outlets have published this data. Why hasn't someone turned it into a csv? Wait - why don't I convert it into a CSV from a news outlet's site? 

### Part 1: Web Scraping using Beautiful Soup
I decided to use the vote count published [here](https://www.nbcnews.com/politics/2020-elections/president-results) on the NBC News Website. 

I loaded the table data into a using the 'tbody' tag, which stands for 'table body'. I then used the 'div' tag, which represents a new row in the table, to separate data from each State/Territory into its own item. 

I then loaded each element of the voting results into its own list: 
- State Name 
- Electoral Votes 
- Percent of Vote Counted
- Biden's Votes
- Trump's Votes

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
url = "https://www.nbcnews.com/politics/2020-elections/president-results"
page = urlopen(url)
html_bytes = page.read()
html = html_bytes.decode("utf-8")
soup = BeautifulSoup(html, 'html.parser')

In [2]:
all_data = soup.find('tbody')
state_data = all_data.find_all('div')
print(len(state_data))

56


In [3]:
print(state_data[1])
# I printed the HTML block to use as a reference for parsing each state's results, so that 
# I wouldn't have to refer back to the original webpage during this process.

<div data-all='{"totalElectoralVotes":3,"percentOfVotesInFormatted":"99%","href":"/politics/2020-elections/alaska-president-results?icid=election_usmap","candidates":[{"lastName":"Trump","firstName":"Donald","totalVotesReceived":189951,"percentageOfVotesReceivedFormatted":"52.8%","totalVotesReceivedFormatted":"189,951","percentageOfVotesReceived":52.8,"isWinner":true,"colorHex":"#DE3535","colorHexValue":"#DE3535"},{"lastName":"Biden","firstName":"Joe","totalVotesReceived":153778,"percentageOfVotesReceivedFormatted":"42.8%","totalVotesReceivedFormatted":"153,778","percentageOfVotesReceived":42.8,"isWinner":false,"colorHex":"#0471E6","colorHexValue":"#0471E6"}],"callStatus":"","raceHasStarted":true,"pollsClose":"1AM","name":"Alaska","isPollsClosed":true}' data-us-state="Alaska"><tr><th scope="row">Alaska</th><td>3</td><td>99%</td><td>Biden 42.8% 153,778 votes</td><td>Trump 52.8% 189,951 votes</td><td><a class="" href="/politics/2020-elections/alaska-president-results?icid=election_usmap"

In [4]:
states = [] 
electoral_votes = []
pct_counted = []
biden = [] 
trump = []
url = [] 
for item in state_data: 
    state_name = item.tr.find('th', scope='row').text
    states.append(state_name)
    
    e_votes = item.tr.find_all('td')[0].text
    electoral_votes.append(e_votes)
    
    percent_in = item.tr.find_all('td')[1].text
    pct_counted.append(percent_in)
    
    biden_votes = item.tr.find_all('td')[2].text
    biden.append(biden_votes)
    
    #trump_votes = item.tr.find_all('td')[3].text
    #trump.append(trump_votes)
    for a in item.find_all('a', href=True):
        state_url = a['href']
    url.append(state_url)       

#### 1.1: Getting Creative

This method worked for all lists except for Trump's results. This likely means that there are inconsistencies in the way the data is stored in each row in the table. 

I verified that the parsing worked for most elements by printing the first result in each list above. I suspected the inconsistency is coming from the territories, and not the US states.

To get a list of Trump's results, I used a ReGex pattern iterating over the table results. I can see that the  length of this list is five less than the length of the other lists. 


In [5]:
print("length:",len(states),"first result:",states[0])
print("length:",len(electoral_votes),"first result:", electoral_votes[0])
print("length:",len(pct_counted),"first result:",pct_counted[0])
print("length:",len(biden),"first result:",biden[0])
print("length:",len(url),"first result:",url[0])

length: 56 first result: Alabama
length: 56 first result: 9
length: 56 first result: 100%
length: 56 first result: Biden 36.6% 849,648 votes
length: 56 first result: /politics/2020-elections/alabama-president-results?icid=election_usmap


In [6]:
import re
trump = all_data.find_all(string=re.compile(r"Trump"))
print(len(trump))

51


### Part Two: Creating the DataFrame

To determine which entries had incomplete information, I read the information into a dataframe and printed the results. I can see that American Samoa, Guam, the Mariana Islands, the Virgin Islands, and Puerto Rico do not have complete information. Interestingly, these territories are not actually present in the table on the original webpage. 

I removed them from my analysis, then added Trump's results as a column in the dataframe.

In [7]:
import pandas as pd
import numpy as np
results_2020 = pd.DataFrame({'State':states,'Electoral Votes':electoral_votes,
                             '% Vote Counted':pct_counted, 'Biden Results':biden,'URL':url})
print(results_2020)

                                           State Electoral Votes  \
0                                        Alabama               9   
1                                         Alaska               3   
2                                        Arizona              11   
3                                       Colorado               9   
4                                        Florida              29   
5                                        Georgia              16   
6                                        Indiana              11   
7                                         Kansas               6   
8                                          Maine               4   
9                                  Massachusetts              11   
10                                     Minnesota              10   
11                                    New Jersey              14   
12                                North Carolina              15   
13                                  North Dakota

In [8]:
results_2020 = results_2020.drop(index = [44,45,46,49,50])
len(results_2020)

51

In [9]:
results_2020['Trump Results'] = trump

### Part 3: Cleaning the DataFrame

Although the data had already come a long way from the HTML format, there was more to do before it would be usable as a CSV. 

The first step I took was removing the '% Vote Counted' column - at the time of scraping this data, all states and territories had either 99% or 100% of votes counted, making this measure unimportant.  

The next step was separating the results for each candidate into distinct columns for vote count and percentage. I ran into a problem, however, when attempting this: I was getting an error. After looking at the results of the column split in a new list, I realized that the data for the US territories was empty, so I removed these from my dataframe. This left only the fifty states and Washington D.C. as my row values. 

In [10]:
results_2020['% Vote Counted'].value_counts()

100%    48
99%      3
Name: % Vote Counted, dtype: int64

In [11]:
results_2020 = results_2020.drop(columns = '% Vote Counted')
results_2020.head()

,State,Electoral Votes,Biden Results,URL,Trump Results
0,Alabama,9,"Biden 36.6% 849,648 votes",/politics/2020-elections/alabama-president-res...,"Trump 62% 1,441,168 votes"
1,Alaska,3,"Biden 42.8% 153,778 votes",/politics/2020-elections/alaska-president-resu...,"Trump 52.8% 189,951 votes"
2,Arizona,11,"Biden 49.4% 1,672,143 votes",/politics/2020-elections/arizona-president-res...,"Trump 49.1% 1,661,686 votes"
3,Colorado,9,"Biden 55.4% 1,804,352 votes",/politics/2020-elections/colorado-president-re...,"Trump 41.9% 1,364,607 votes"
4,Florida,29,"Biden 47.9% 5,297,045 votes",/politics/2020-elections/florida-president-res...,"Trump 51.2% 5,668,731 votes"


#### 3.1: Splitting Columns

After cleaning the territories from my data, I was able to successfully split the data for each candidate's votes into three distinct columns. I then removed the original columns from the dataframe, as well as the ones which just contained the values 'Biden' or 'Trump'. 

Then, I cleaned the URL column. Because of the way that the data was stored in the HTML file, the base URL https://www.nbcnews.com/ was missing, so I concatenated it with the values in the URL column. 

In [12]:
results_2020[['Biden','Biden Vote Share','Biden Vote Count']]= results_2020['Biden Results'].str.split(" ",n=2,expand = True)
results_2020[['Trump','Trump Vote Share','Trump Vote Count']]= results_2020['Trump Results'].str.split(" ",n=2,expand = True)
results_2020 = results_2020.drop(columns = ['Trump','Biden','Trump Results','Biden Results'])
results_2020['URL']='https://www.nbcnews.com'+results_2020['URL']
results_2020.head()

,State,Electoral Votes,URL,Biden Vote Share,Biden Vote Count,Trump Vote Share,Trump Vote Count
0,Alabama,9,https://www.nbcnews.com/politics/2020-election...,36.6%,"849,648 votes",62%,"1,441,168 votes"
1,Alaska,3,https://www.nbcnews.com/politics/2020-election...,42.8%,"153,778 votes",52.8%,"189,951 votes"
2,Arizona,11,https://www.nbcnews.com/politics/2020-election...,49.4%,"1,672,143 votes",49.1%,"1,661,686 votes"
3,Colorado,9,https://www.nbcnews.com/politics/2020-election...,55.4%,"1,804,352 votes",41.9%,"1,364,607 votes"
4,Florida,29,https://www.nbcnews.com/politics/2020-election...,47.9%,"5,297,045 votes",51.2%,"5,668,731 votes"


#### 3.2: Cleaning Numeric Data

All of the columns at this point are 'object' dtype, but most of them hold numeric values. 

First, I cleaned the percentage data by eliminating the percent sign, converting each datapoint to a float data type, and dividing the number by 100 to get the raw ratio. 

I used a similar function on the vote count columns. Here, I eliminated the word 'votes' and got rid of the commas. I then converted the data to dtype integer. 

Finally, I converted the 'Electoral Votes' Column to dtype integer as well. 

In [13]:
percent_cols = ['Biden Vote Share','Trump Vote Share']
def clean_percent(item): 
    item = item.replace('%',"")
    item = float(item)
    item = item/100 
    return item
for col in percent_cols: 
    results_2020[col] = results_2020[col].apply(clean_percent)
results_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 55
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State             51 non-null     object 
 1   Electoral Votes   51 non-null     object 
 2   URL               51 non-null     object 
 3   Biden Vote Share  51 non-null     float64
 4   Biden Vote Count  51 non-null     object 
 5   Trump Vote Share  51 non-null     float64
 6   Trump Vote Count  51 non-null     object 
dtypes: float64(2), object(5)
memory usage: 3.2+ KB


In [14]:
count_cols = ['Biden Vote Count', 'Trump Vote Count']
def clean_count(item):
    item = item.replace(' votes',"")
    item = item.replace(',',"")
    item = int(item)
    return item
for col in count_cols: 
    results_2020[col] = results_2020[col].apply(clean_count)
results_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 55
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State             51 non-null     object 
 1   Electoral Votes   51 non-null     object 
 2   URL               51 non-null     object 
 3   Biden Vote Share  51 non-null     float64
 4   Biden Vote Count  51 non-null     int64  
 5   Trump Vote Share  51 non-null     float64
 6   Trump Vote Count  51 non-null     int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 3.2+ KB


In [15]:
results_2020['Electoral Votes'] = results_2020['Electoral Votes'].astype(int)
results_2020.head()

,State,Electoral Votes,URL,Biden Vote Share,Biden Vote Count,Trump Vote Share,Trump Vote Count
0,Alabama,9,https://www.nbcnews.com/politics/2020-election...,0.366,849648,0.620,1441168
1,Alaska,3,https://www.nbcnews.com/politics/2020-election...,0.428,153778,0.528,189951
2,Arizona,11,https://www.nbcnews.com/politics/2020-election...,0.494,1672143,0.491,1661686
3,Colorado,9,https://www.nbcnews.com/politics/2020-election...,0.554,1804352,0.419,1364607
4,Florida,29,https://www.nbcnews.com/politics/2020-election...,0.479,5297045,0.512,5668731


#### 3.3: Sorting Data

The data is really close to being ready to export, but I realized that the states are in a random order. I sorted the data alphabetically by state, the format in which one would expect this data to be displayed.

In [16]:
results_2020 = results_2020.sort_values(by='State')
results_2020.reset_index(drop=True,inplace = True,)
results_2020.head()

,State,Electoral Votes,URL,Biden Vote Share,Biden Vote Count,Trump Vote Share,Trump Vote Count
0,Alabama,9,https://www.nbcnews.com/politics/2020-election...,0.366,849648,0.620,1441168
1,Alaska,3,https://www.nbcnews.com/politics/2020-election...,0.428,153778,0.528,189951
2,Arizona,11,https://www.nbcnews.com/politics/2020-election...,0.494,1672143,0.491,1661686
3,Arkansas,6,https://www.nbcnews.com/politics/2020-election...,0.348,423932,0.624,760647
4,California,55,https://www.nbcnews.com/politics/2020-election...,0.635,11109764,0.343,6005961


### Step 4: Exporting the Data

Now that I am happy with how my data looks, I can export my data to a CSV file. 
I simply used the pandas.to_csv( ) function for this. 

In [17]:
results_2020.to_csv('/Users/austenbrockman/Desktop/python/NBC 2020 Election Results.csv', index=False)